EE478 Homework 1
----------------

Hansem Ro




Problem 1
---------

1-of-16 decoder:
- inputs: $A[3:0], \bar{A}[3:0]$
- outputs: $word_0, word_1, ..., word_{15}$

Decoder logic:
$
word_0 = \bar{A}[3]\&\bar{A}[2]\&\bar{A}[1]\&\bar{A}[0]\\
word_1= \bar{A}[3]\&\bar{A}[2]\&\bar{A}[1]\&A[0]\\
...\\
word_{15} = A[3]\&A[2]\&A[1]\&A[0]
$

Assumptions:
- 0.2fF/um of wire
- 2 std-cell vertical pitch = 3.6um
- direct decoding
- inverter input cap = 1fF
- output load cap = (4fF * 32bits) = 128fF


In [6]:
# Install numpy and cvxpy
import numpy as np
import circuit_size_solver as solver


In [3]:
# Testing cvxpy: Book example 4.15 (W&H)
# By: Hansem Ro <hansem7@uw.edu>
#
#                 _____
#--------------->|NAND2|
#--[Inv(x=1)]-+->|(x2) |---+
#             |  |_____|   |     _____
#             +->|NOR2 |   +--->|NOR3 |
#--------------->|(x3) |------->|(x4) |----+----[Inv(x5)]----+
#                |_____|   +--->|_____|    |                 |
#                          |              ===c4=10          ===c5=12
#--------------------------+               |                 |
#                                          v                 v
#
# Expected solutions:
#  Minimum arrival time = 23.44
#  x2 = 1.62
#  x3 = 1.62
#  x4 = 3.37
#  x5 = 6.35

# Step 1: generate delay expressions for each gate
#
# d = f + p = g*h + p = g*(Cout/Cin) + p = (Cout/x) + p
# Cin = g*x

x2 = cp.Variable(pos=True, name="x2")
x3 = cp.Variable(pos=True, name="x3")
x4 = cp.Variable(pos=True, name="x4")
x5 = cp.Variable(pos=True, name="x5")

c4 = 10
c5 = 12

# INV (x=1): branches to NAND2 and NOR2
c_inv_nand2 = cp.multiply(g_nand(2),x2)
c_inv_nor2 = cp.multiply(g_nor(2),x3)
d_inv1 = p_inv() + (c_inv_nand2+c_inv_nor2)/g_inv()
print("d_inv1 = ", d_inv1)

# NAND2 (x2): connects to NOR3
c_nor3 = cp.multiply(g_nor(3),x4)
d_nand2 = p_nand(2) + (c_nor3)/x2
print("d_nand2 = ", d_nand2)

# NOR2 (x3): connects to NOR3
d_nor2 = p_nor(2) + (c_nor3)/x3
print("d_nor2 = ", d_nor2)

# NOR3 (x4): connects to c4 and Inv (x5)
c_nor3_inv = cp.multiply(g_inv(),x5)
c_nor3_c4 = c4
d_nor3 = p_nor(3) + (c_nor3_inv + c_nor3_c4)/x4
print("d_nor3 = ", d_nor3)

# INV (x5): connects to c5
c_inv_c5 = c5
d_inv2 = p_inv() + (c_inv_c5)/x5
print("d_inv2 = ", d_inv2)

# Step 2: Express as arrival time
a_inv1 = d_inv1
a_nand2 = a_inv1 + d_nand2
a_nor2 = a_inv1 + d_nor2
a_nor3 = cp.maximum(a_nand2,a_nor2) + d_nor3
a_inv2 = a_nor3 + d_inv2
print("a_inv1 = ", a_inv1)
print("a_nand2 = ", a_nand2)
print("a_nor2 = ", a_nor2)
print("a_nor3 = ", a_nor3)
print("a_inv2 = ", a_inv2)

# Step 3: Minimize latest arrival time (a_inv2)

problem = cp.Problem(cp.Minimize(a_inv2))
print(problem.is_dgp(dpp=True))
print(problem.solve(gp=True))
print("x2 = ", x2.value)
print("x3 = ", x3.value)
print("x4 = ", x4.value)
print("x5 = ", x5.value)



d_inv1 =  1.0 + (1.3333333333333333 @ x2 + 1.6666666666666667 @ x3) / 1.0
d_nand2 =  2.0 + 2.3333333333333335 @ x4 / x2
d_nor2 =  2.0 + 2.3333333333333335 @ x4 / x3
d_nor3 =  3.0 + (1.0 @ x5 + 10.0) / x4
d_inv2 =  1.0 + 12.0 / x5
a_inv1 =  1.0 + (1.3333333333333333 @ x2 + 1.6666666666666667 @ x3) / 1.0
a_nand2 =  1.0 + (1.3333333333333333 @ x2 + 1.6666666666666667 @ x3) / 1.0 + 2.0 + 2.3333333333333335 @ x4 / x2
a_nor2 =  1.0 + (1.3333333333333333 @ x2 + 1.6666666666666667 @ x3) / 1.0 + 2.0 + 2.3333333333333335 @ x4 / x3
a_nor3 =  maximum(1.0 + (1.3333333333333333 @ x2 + 1.6666666666666667 @ x3) / 1.0 + 2.0 + 2.3333333333333335 @ x4 / x2, 1.0 + (1.3333333333333333 @ x2 + 1.6666666666666667 @ x3) / 1.0 + 2.0 + 2.3333333333333335 @ x4 / x3) + 3.0 + (1.0 @ x5 + 10.0) / x4
a_inv2 =  maximum(1.0 + (1.3333333333333333 @ x2 + 1.6666666666666667 @ x3) / 1.0 + 2.0 + 2.3333333333333335 @ x4 / x2, 1.0 + (1.3333333333333333 @ x2 + 1.6666666666666667 @ x3) / 1.0 + 2.0 + 2.3333333333333335 @ x4 / x3

In [7]:
# Testing cvxpy: Book example 4.15 (W&H)
# By: Hansem Ro <hansem7@uw.edu>
#
#                   _____
# A--------------->|NAND2|
# B--[Inv(x=1)]-+->|(x2) |---+
#               |  |_____|   |     _____
#               +->|NOR2 |   +--->|NOR3 |
# C--------------->|(x3) |------->|(x4) |----+----[Inv(x5)]----+
#                  |_____|   +--->|_____|    |                 |
#                            |              ===c4=10          ===c5=12
# D--------------------------+               |                 |
#                                            v                 v
#
# Expected solutions:
#  Minimum arrival time = 23.44
#  x2 = 1.62
#  x3 = 1.62
#  x4 = 3.37
#  x5 = 6.35

# Using logical_unit class
inputs = np.array(["A", "B", "C", "D"])
top = solver.top_module(inputs)
top.add_inv("B", "net_inv1", drive=1, name="inv1")
top.add_unit(np.array(["A", "net_inv1"]), "net_nand2", type="nand", drive="x2", name="nand2")
top.add_unit(np.array(["net_inv1", "C"]), "net_nor2", type="nor", drive="x3", name="nor2")
top.add_unit(np.array(["net_nand2", "net_nor2", "D"]), "net_nor3", type="nor", drive="x4")
top.add_cap("net_nor3", "net_c4", 10, name="c4")
top.add_inv("net_nor3", "net_inv2", drive="x5", name="inv2")
top.add_cap("net_inv2", "net_c5", 12, name="c5")
# top.print_props()
# print(top.get_cap("net_inv1"))
top.solve()

Creating top level module
True
Minimum arrival time:  23.455298867010317
name:  nor2  drive:  1.6186554353103
name:  inv1  drive:  1.0
name:  inv2  drive:  6.357952425532625
name:  nand2  drive:  1.6186554375276394
name:  nor3_net_nor3  drive:  3.368629814288384
